# Agent to take follow up actions after analysis issue fix

This is an agent that uses "Additional Information" from fix generation agent to make further changes.

For faster prototyping, we use LangGraph for implementing this.

We create a simple network of agents - FixGenerationAgent, FileOperationsAgent. 

The _FixGenerationAgent_ is the same as what we use today to fix analyzer issues. _FileOperationsAgent_ is a new agent we introduce that takes follow up actions. The follow up actions include updating another file, removing a file, or creating a new file. For each operation, we have a tool defined.

![Diagram](./screenshots/Tool.png)

# Fix Generation Agent

This is same as the existing agent we use to fix analysis issues.

We are using a few hardcoded examples of analysis issues to generate fixes.

Before proceeding, make sure you are using Kai venv to run cells in this notebook.

We need to install `langgraph` module, run following cell:

In [ ]:
%load_ext dotenv
%dotenv
%pip install langgraph

In [ ]:
# THIS CELL CONTAINS SOME COMMON FUNCTIONS WE WILL USE, RUN THIS CELL BEFORE MOVING FORWARD

import re

def parse_llm_response(message: str) -> tuple[str, str, str]:
    lines_of_output = message.splitlines()
    in_source_file = False
    in_reasoning = False
    in_additional_details = False
    source_file = ""
    reasoning = ""
    additional_details = ""
    for line in lines_of_output:
        # trunk-ignore(cspell/error)
        if re.match(r"(?:##|\*\*)\s+[Rr]easoning", line.strip()):
            in_reasoning = True
            in_source_file = False
            in_additional_details = False
            continue
        # trunk-ignore(cspell/error)
        if re.match(r"(?:##|\*\*)\s+[Uu]pdated.*[Ff]ile", line.strip()):
            in_source_file = True
            in_reasoning = False
            in_additional_details = False
            continue
        # trunk-ignore(cspell/error)
        if re.match(r"(?:##|\*\*)\s+[Aa]dditional\s+[Ii]nformation", line.strip()):
            in_reasoning = False
            in_source_file = False
            in_additional_details = True
            continue
        if in_source_file:
            if re.match(r"```(?:\w*)", line):
                continue
            source_file = "\n".join([source_file, line])
        if in_reasoning:
            reasoning = "\n".join([reasoning, line])
        if in_additional_details:
            additional_details = "\n".join([additional_details, line])
    return source_file, reasoning, additional_details

We will create the analysis fix agent and generate a few example fixes to be used with file operation agents. 

The example issues are taken from Coolstore application analysis and can be found in [example_prompts](./example_prompts/) directory.

Running the following cell will generate analysis fixes for all examples and store them in variables for later use.

In [ ]:
from pathlib import Path
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.prebuilt import create_react_agent

llm = ChatOpenAI(model="gpt-4o")

# fix generation agent
fix_gen_agent = create_react_agent(
    model=llm,
    name="fix_gen_agent",
    prompt="You are an experienced Java developer, who specializes in migrating code from Java EE to Quarkus",
    tools=[],
)

raw_response_ex1 = fix_gen_agent.invoke({
    "messages": [
        HumanMessage(content=Path("example_prompts", "inventory_notification_java", "jms-to-reactive-quarkus-00050").read_text()),
    ]
})

In [16]:
_, _, additional_info_ex1 = parse_llm_response(raw_response_ex1["messages"][-1].content)
print("We will use following additional info for Example-1:")
print(additional_info_ex1)

We will use following additional info for Example-1:

- **Dependencies Update in POM:** Ensure to add the `quarkus-smallrye-reactive-messaging` dependency to your `pom.xml` file:

```xml
<dependency>
    <groupId>io.quarkus</groupId>
    <artifactId>quarkus-smallrye-reactive-messaging</artifactId>
</dependency>
```

- **Configuration:** The source channel name "orders" should be configured in `application.properties` or `application.yaml` to ensure integration with the message broker.
  
- With these updates, the service is better integrated in the Quarkus ecosystem, leveraging modern practices for message-driven architectures.


## File Operations Agent

This is the agent that will take "Additional Information" generated by previous agent as input and take actions if there is anything important in additional information. 

The actions include:
 * updating an existing file (which may not be same as the original file we fixed)
 * deleting a file
 * creating a new file

The agent will use following tools to take the above options:
 * find: Finds relevant files in the project
 * read: Reads a file in the project
 * write: Writes content to a file in the project
 * rm: Removes a file from the project

In the following cell, we define these tools and the agent.

The [project](./project/) directory is the base of the project and will be used by the agent to perform file operations.

In [21]:
import os
import glob
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from typing import Annotated

@tool
def find(file_name_pattern: Annotated[str, "Glob pattern to match file names"]) -> Annotated[str, "Absolute paths of the files found separated by newlines"]:
    """Given a glob pattern, searches for files matching the pattern in project and returns absolute paths of the files

    Returns:
        str: Newline separated list of filepaths
    """
    try:
        matching_files = glob.glob("./project/" + file_name_pattern, recursive=True)
    except Exception as e:
        return f"Failed to search for files because of error - {e}"
    absolute_paths = [os.path.abspath(file) for file in matching_files]
    if not absolute_paths:
        return f"No files found matching pattern {file_name_pattern}"
    return "\n".join(absolute_paths)

@tool
def read(file_name: Annotated[str, "Absolute path to the file to read"]):
    """Given an absolute path to a file, reads the file and returns content"""
    if os.path.exists(file_name):
        contents = ""
        with open(file_name, "r") as f:
            contents = f.read()
        return f"Here are the file contents:\n```{contents}```"
    return f"File {file_name} does not exist"

@tool
def write(file_name: Annotated[str, "Absolute path to the file"], content: Annotated[str, "Content to write to the file"]):
    """Given an absolute path to a file and the content, writes content to the file. If the file doesn't exist, creates a new file.
    If the directories dont exist, creates all parent directories."""
    try:
        os.makedirs(os.path.dirname(file_name), exist_ok=True)
        with open(file_name, "w+") as f:
            f.write(content)
        return f"Successfully wrote contents to file {file_name}"
    except Exception as e:
        return f"Failed to write to file with error {e}"

@tool
def rm(file: Annotated[str, "Full path of the file to remove"]):
    """Removes file"""
    if os.path.exists(file):
        return "Removed the file"
    return f"File {file} does not exist"

file_ops_agent = create_react_agent(
    model=llm,
    tools=[find, read, write, rm],
    prompt="""You are an expert Java developer. You are asked to make changes in a Java project.
You can either update an existing file, delete a file, or create a new file with relevant content.
To update or delete an existing file, search for the file to update using a glob pattern, and determine which file to update from the files found.
To update an existing file, make sure you read the file to get existing contents, and then write to the file updated content.
If a file doesn't exist and needs to be created, create that file.
Always use absolute file paths to make changes to files. You are given tools to perform file operations. Use the tools, and make the changes needed.
Use your best judgement to solve the issues described.
""",
)

for state in file_ops_agent.stream(
    {"messages": [HumanMessage(content=additional_info_ex1)]},
    stream_mode=["updates"]
):
    print(state)

('updates', {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MPVUNqWJ6H6VyQvbNLl6LsnB', 'function': {'arguments': '{"file_name_pattern":"pom.xml"}', 'name': 'find'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 499, 'total_tokens': 516, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-09a71789-601d-4be5-ab54-e554a1137879-0', tool_calls=[{'name': 'find', 'args': {'file_name_pattern': 'pom.xml'}, 'id': 'call_MPVUNqWJ6H6VyQvbNLl6LsnB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 499, 'output_tokens': 17, 'total_tokens': 516, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'o

## Results

In the above output, you can see that the model searched for `pom.xml` file and added the dependency. 

It also searched for `application.properties` file but it wasn't found, so it created a new file at path `src/main/resources` which is the correct path for Quarkus applications. 

This proves the possibility of updating another file from the project, AND creating a new file.

I have attached screenshots of the results from my environment below.

![pom_xml_update](./screenshots/pom_xml_update.png)

![application_properties_creation](./screenshots/application_properties_creation.png)